In [ ]:
import numpy as np
import matplotlib.pyplot as plt
data = np.load('flatland_train.npz')
X = data['X']
y = data['y']

y[y != 0] -= 2    # Correct labels so that triangle is mapped to class 1
X = X / 255.      # Scale down to range [0, 1]



In [ ]:
# Construct and train your model (don't forget train/test split and other tricks)
from tensorflow import keras



In [ ]:
#Sukonstruojame train/test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3, random_state=42)

In [ ]:
#Simple NN
model=keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=(50,50)))
model.add(keras.layers.Dense(1000,activation='relu'))
model.add(keras.layers.Dense(50,activation='relu'))
model.add(keras.layers.Dense(5,activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',optimizer='SGD', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 2500)              0         
                                                                 
 dense (Dense)               (None, 1000)              2501000   
                                                                 
 dense_1 (Dense)             (None, 50)                50050     
                                                                 
 dense_2 (Dense)             (None, 5)                 255       
                                                                 
Total params: 2,551,305
Trainable params: 2,551,305
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Pakeičiame X figūrą, kad galėtume naudoti Conv2D funkciją
X_train=X_train.reshape(X_train.shape[0],50,50,1)
X_test=X_test.reshape(X_test.shape[0],50,50,1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Flatten, Conv2D, BatchNormalization, \
                                    Activation, Dropout, MaxPooling2D

In [ ]:
#CNN
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=[50, 50, 1]))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(63, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Adam(),
              metrics=["accuracy"])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 48, 48, 32)        320       
                                                                 
 conv2d_3 (Conv2D)           (None, 46, 46, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 64)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 23, 23, 64)        0         
                                                                 
 flatten_3 (Flatten)         (None, 33856)             0         
                                                                 
 dense_8 (Dense)             (None, 63)                2132991   
                                                      

In [ ]:
# Geriausiai veikiantis modelis su dropout ir maxpooling funkcijom, kurios pagerina modelio veikimą.

# Adding new trainable Dense layers
inputs = keras.layers.Input(shape=(50, 50, 1))
# Output shape for sure is problem dependant, but this should give you the idea
output = keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=[50, 50, 1])(inputs)
output = keras.layers.BatchNormalization(axis=-1) (output)   # Axis -1 is always the features axis
output = keras.layers.Activation("relu")(output)
output = keras.layers.MaxPooling2D(pool_size=(2, 2))(output)
output = keras.layers.Conv2D(64, (3, 3), activation='relu')(output)
output = keras.layers.BatchNormalization(axis=-1) (output)   # Axis -1 is always the features axis
output = keras.layers.Activation("relu")(output)
output = keras.layers.MaxPooling2D(pool_size=(2, 2))(output)
output = keras.layers.Flatten()(output)
output = keras.layers.Dropout(0.4)(output)
output = keras.layers.Dense(250, activation='relu')(output)
output = keras.layers.Dropout(0.6)(output)
output = keras.layers.Dense(150, activation='relu')(output)
output = keras.layers.Dropout(0.6)(output)
output = keras.layers.Dense(5, activation='softmax')(output)
# Build the model
model = keras.models.Model(inputs=inputs, outputs=output)
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=["accuracy"])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 50, 50, 1)]       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 48, 48, 32)        320       
                                                                 
 batch_normalization_2 (Batc  (None, 48, 48, 32)       128       
 hNormalization)                                                 
                                                                 
 activation_2 (Activation)   (None, 48, 48, 32)        0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 24, 24, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 22, 22, 64)        1849

In [ ]:
#Treniruojame modelį
model.fit(X_train, y_train, epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
219/219 [==============================] - 3s 9ms/step - loss: 2.0368 - accuracy: 0.2656 - val_loss: 1.5914 - val_accuracy: 0.2903
Epoch 2/100
219/219 [==============================] - 1s 7ms/step - loss: 1.4873 - accuracy: 0.3351 - val_loss: 1.5327 - val_accuracy: 0.4027
Epoch 3/100
219/219 [==============================] - 2s 7ms/step - loss: 1.4166 - accuracy: 0.3906 - val_loss: 1.3965 - val_accuracy: 0.4417
Epoch 4/100
219/219 [==============================] - 2s 7ms/step - loss: 1.3356 - accuracy: 0.4270 - val_loss: 1.1946 - val_accuracy: 0.5253
Epoch 5/100
219/219 [==============================] - 2s 7ms/step - loss: 1.2278 - accuracy: 0.4726 - val_loss: 1.1727 - val_accuracy: 0.5027
Epoch 6/100
219/219 [==============================] - 2s 7ms/step - loss: 1.2045 - accuracy: 0.4800 - val_loss: 1.0898 - val_accuracy: 0.5723
Epoch 7/100
219/219 [==============================] - 2s 7ms/step - loss: 1.1379 - accuracy: 0.5189 - val_loss: 1.0209 - val_accuracy: 0.6150

In [ ]:
# Save the model and upload to git
model.save('model.h5')